<a href="https://colab.research.google.com/github/LSschenke/beck-leitTest/blob/main/beck_leits%C3%A4tze_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:




















# Testversion -> Prof Schenke 

############
### Packages
############
import time # Package Time
import random # Package Zufallszahlen
import win32com.client # Package zum Lesen von Office-Files importieren
from selenium import webdriver # Package selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import os # Package os
import glob # Package glob
import shutil
import sys
from winsound import Beep


##############
### Funktionen
##############
def startbeck():
    ### Beck-Online-Seite öffnen und Benutzername und Passwort eingeben
    # open beck-online homepage
    driver.get('https://beck-online.beck.de/Home')
    time.sleep(4+2*random.random())

    # maximize chrome window
    driver.maximize_window()

    # click cookie bottom
    searchAndClick('//*[@id="uc-btn-accept-banner"]')
    time.sleep (3+2*random.random())

    # enter username
    benutzername = driver.find_element_by_xpath('//*[@id="username"]')
    benutzername.send_keys('Ralf.Schenke')
    time.sleep (2+3*random.random())

    # enter password
    password = driver.find_element_by_xpath('//*[@id="passwd"]')
    password.send_keys('105255Bo')
    time.sleep (2+3*random.random())

    # click anmeldung
    searchAndClick('//*[@id="login-button"]')

def AuslesenExcel(x, alldata):
    ### liest aus dem Excel-File die Zeitschrift, das Jahr und die Seite aus
    # und gibt verschiedene strings, u.a. mit der kompletten Fundstelle zurück

    key = sheetFundstelle.Cells(x, 2)
    print('key:', type(key), ' - ', key)
    key = int(key)
    key = str(key)
    autor = sheetFundstelle.Cells(x, 3)
    autor = str(autor)
    fundstelle = sheetFundstelle.Cells(x, 4)
    fundstelle = str(fundstelle) # fundstelle in string umwandeln
    jahr = sheetFundstelle.Cells(x, 5)
    jahr = int(jahr)
    jahr = str(jahr)
    seite = sheetFundstelle.Cells(x, 6)
    seite = int(seite)
    seite = str(seite)
    kurztitel = sheetFundstelle.Cells(x, 7)
    kurztitel = str(kurztitel)
    suche = fundstelle+" "+jahr+", "+seite
    return suche, key, autor, fundstelle, jahr, seite, kurztitel

def searchAndClick(suche, sprachausgabe=''):
    if sprachausgabe=='':
        print('Keine Sprachausgabe')
    else:
        speaker.speak(sprachausgabe)
    string = "window.scrollTo(0, x)"
    a=200
    grenzwert = 1000
    while a<grenzwert:
        y=str(a)
        eingabe=string.replace("x", y)
        driver.execute_script(eingabe)
        time.sleep(1)
        Beep(500, 500)
        try:
            elem = driver.find_element_by_xpath(suche)
            elem.click()
            a = grenzwert
        except NoSuchElementException:  # spelling error making this code not work as expected
            a = a + 100
            pass

def ladeAusBeckOnline(x):
### gibt den Eintrag in das Suchfenster auf Beck-OK ein, startet die Suche, speichert das Ergebnis


### Funktion welche Detailsuche nur auf Rechtsprechung stellt ? 

### Detailsuche iD Element: "DetailSearchLink"

### Detailsuche Rechtsprechung Checkbox ID: "chkRspr"


    searchbox = driver.find_element_by_xpath('//*[@id="words"]')
    time.sleep(5 + 2 * random.random())
    searchbox.send_keys(Keys.CONTROL, 'a')  # Eintrag im Suchfenster löschen
    searchbox.send_keys(Keys.BACKSPACE)
    searchbox.send_keys(suche)
    time.sleep(3 + 3 * random.random())
    searchAndClick('/html/body/div[3]/form/div/div[2]/div[2]/div[2]/div[2]/div[2]/label')

    ### Speichert den Eintrag ab
    time.sleep(2 + 2 * random.random())
    # sucht und klickt den Druck- und Export-button
    searchAndClick('/html/body/div[4]/div[3]/div[2]/div[2]/div[3]/div[1]/div[1]/span', 'Druck- und Export')
    time.sleep(5 + 2 * random.random())
    # Clicke auf Weitere Optionen
    searchAndClick('/html/body/div[4]/div[3]/div[2]/div[2]/div[3]/div[2]/form/div[2]/a', 'Weitere Optionen')
    time.sleep(5 + 2 * random.random())
    # Exportieren nach word
    searchAndClick('//*[@id="wordExportButton"]', 'Exportiere nach word')
    # Scrolle wieder zum Seitenanfang zurück
    time.sleep(5 + 2 * random.random())
    driver.execute_script("window.scrollTo(0, 10)")


#################
### Hauptprogramm
#################
# Dateien werden in downloadfolder zwischengespeichert
#
#
speaker = win32com.client.Dispatch("SAPI.SpVoice")
#speaker.speak("Jetzt geht es los")

# Selenium initialisieren
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
startbeck() # open beck-online homepage and enter username plus password

### aktuelles Excel-File in die Variable wb laden
# Wichtig: Es darf nur ein Excel-File geöffnet sein
ExcelApp = win32com.client.Dispatch("Excel.Application")
ExcelApp.Visible = True
time.sleep(2)
wb = ExcelApp.Workbooks(1)



### Zugriff auf Liste mit Beiträgen, die heruntergeladen werden sollen
sheet_temp = wb.Worksheets('Fundstellen')
sheetFundstelle = sheet_temp.UsedRange
print(sheetFundstelle)
# Get number of rows used on active sheet
getNumRows = sheetFundstelle.Rows.Count

### Zugriff auf Liste mit Download- und Zielordnern
sheet_temp = wb.Worksheets('Inbox')
sheetInbox = sheet_temp.UsedRange
print(sheetInbox)
downloadFolder = str(sheetInbox.Cells(1, 2))
inboxFolder = str(sheetInbox.Cells(1, 9))
# Aktuelles directory auf downloadFolder einstellen
os.chdir(downloadFolder)

getNumRowsFolder = sheetInbox.Rows.Count

### Schlüssel auslesen aus excel in dictionary inbox
inboxNr = sheetInbox.Range(sheetInbox.Cells(4, 1), sheetInbox.Cells(getNumRowsFolder, 2))
inbox = {} # dictionary definieren
loop = 1
while True:
    #print(inboxNr.Cells(loop, 1))
    if str(inboxNr.Cells(loop, 1)) == 'None':
        break
    nummer = str(int(inboxNr.Cells(loop, 1)))
    ordner = str(inboxNr.Cells(loop, 2))
    inbox.update({nummer : ordner})
    loop += 1

print(inbox)

#sys.exit()

### Schleife mit Durchlauf
for x in range(2, getNumRows+1):

    if str(sheetFundstelle.Cells(x, 1)) == 'None':
        # Wird nur ausgeführt, wenn die ausgelesene Zelle leer ist

        # liest den Eintrag aus dem Excel-File in Reihe x aus
        suche, key, autor, fundstelle, jahr, seite, kurztitel = AuslesenExcel(x, sheetFundstelle) # Einträge beginnen erst in Zeile 4

        # Lädt den Beitrag aus Beck-Online in das Download-Verzeichnis runter
        ladeAusBeckOnline(suche) # z.B. NVwZ 2020, 1

        # Identifiziert im Download-Ordner die letzte heruntergeladene Datei

        list_of_files = glob.glob(downloadFolder+'\\*')  # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)

        Zielort = inboxFolder + "\\"
        if key in inbox:
            Zielort = Zielort + inbox[key] + "\\"

        neuer_name = Zielort + autor + ", " + fundstelle + " " + jahr + ", " + seite + " - "+ kurztitel+".docx"
        print('latest_file: ', latest_file)
        print('neuer_name: ', neuer_name)

        # Verschiebe die Datei und benenne diese um
        shutil.move(latest_file, neuer_name)

        # Wert X in Zelle (x, 1) eintragen, um die Erledigung des Downloads zu markieren
        sheetFundstelle.Cells(x, 1).Value = 'x'




    


ModuleNotFoundError: ignored